In [ ]:
!pip install tensorflow
!pip install datasets
!pip install transformers

In [ ]:
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

inputs = tokenizer(["Hello world!", "How are you"], padding=True, truncation=True, return_tensors="pt")
print(inputs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102],
        [ 101, 2129, 2024, 2017,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


In [ ]:
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102],
        [ 101, 2129, 2024, 2017,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}

In [ ]:
emotions = load_dataset("SetFit/emotion")

README.md:   0%|          | 0.00/194 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/276k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/279k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
emotions['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0, 'label_text': 'sadness'}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define tokenization function
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# Apply tokenizer to dataset
emotions_encoded = emotions.map(tokenize_batch, batched=True)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
!pip install peft torch

In [ ]:
train_data = emotions['train']
test_data = emotions['test']

#apply tokenizer to dataset
train_dataset = train_data.map(tokenize_batch, batched=True)
test_dataset = test_data.map(tokenize_batch, batched=True)

#convet to PyTorch formate
train_dataset.set_format(type = "torch", columns = ["input_ids", "attention_mask", "label"])
test_dataset.set_format(type = "torch", columns = ["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

# Define the model for fine-tuning
# The number of labels is 6 for the emotion dataset
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Import necessary libraries for LoRA fine-tuning
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Define LoRA Configuration
# LoRA (Low-Rank Adaptation) allows efficient fine-tuning by adding small trainable matrices
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation dimension - controls the rank of adaptation matrices
          # Lower values (4-16) = fewer parameters, faster training, less capacity
          # Higher values = more parameters, slower training, more capacity

    lora_alpha=32,  # Scaling factor for LoRA weights
                   # Typically set to 2-4x the rank value
                   # Controls how much the LoRA adaptation affects the original weights

    lora_dropout=0.05,  # Dropout rate applied to LoRA layers
                       # Helps prevent overfitting during fine-tuning
                       # Common values: 0.05 to 0.1

    target_modules=["query", "value"]  # Apply LoRA to self-attention layers only
                                      # "query" and "value" are the most effective for most tasks
                                      # Could also include "key", "dense", "gate_proj", etc.
)

# Prepare model for LoRA fine-tuning
# This step prepares the base model for efficient training with quantization support
model = prepare_model_for_kbit_training(model)

# Convert model into LoRA-enabled model
# This wraps the original model and adds LoRA adapters to specified modules
peft_model = get_peft_model(model, lora_config)

# Print trainable parameters summary
# This shows how many parameters will actually be trained vs total parameters
peft_model.print_trainable_parameters()

# Output interpretation:
# trainable params: 294,912 - Only these parameters will be updated during training
# all params: 109,781,766 - Total parameters in the entire model
# trainable%: 0.2686 - Percentage of parameters being fine-tuned (very efficient!)

# Key Benefits of this LoRA setup:
# 1. Memory Efficient: Only ~0.27% of parameters need gradients
# 2. Fast Training: Significantly faster than full fine-tuning
# 3. Storage Efficient: Only need to save the small LoRA weights
# 4. Preserve Base Model: Original model weights remain frozen
# 5. Easy Deployment: Can easily switch between different LoRA adapters

trainable params: 294,912 || all params: 109,781,766 || trainable%: 0.2686


In [ ]:
#CODE explain with the help of Claude
# ============================================================================
# TRAINING SETUP: Configure and Execute LoRA Fine-tuning
# ============================================================================

# Import required training components from transformers library
from transformers import TrainingArguments, Trainer

# ============================================================================
# TRAINING ARGUMENTS: Define how the training should happen
# ============================================================================

training_args = TrainingArguments(
    # OUTPUT CONFIGURATION
    output_dir="./model_checkpoint",  # Directory where model checkpoints will be saved
                                     # Creates folder structure for versioning and recovery

    # TRAINING SCHEDULE
    num_train_epochs=1,              # Train for 1 complete pass through the dataset
                                    # More epochs = better learning but risk of overfitting
                                    # Start with 1-3 epochs for LoRA fine-tuning

    per_device_train_batch_size=16,  # Process 16 samples per GPU/device at once
                                    # Higher = faster training but more memory usage
                                    # Adjust based on your GPU VRAM (4-32 typical range)

    # EVALUATION SETTINGS
    eval_strategy="epoch",      # When to run evaluation on validation set
                                     # Options: "no", "steps", "epoch"
                                     # "epoch" = evaluate after each training epoch

    save_strategy="epoch",           # When to save model checkpoints
                                    # "epoch" = save after each epoch for recovery
                                    # Prevents losing progress if training crashes

    # LOGGING AND MONITORING
    logging_steps=10,                # Log training metrics (loss, learning rate) every 10 steps
                                    # Lower = more frequent logging, higher = less verbose
                                    # Helps monitor training progress and detect issues

    load_best_model_at_end=True,     # Automatically load the best performing checkpoint at end
                                    # Based on evaluation metric (usually validation loss)
                                    # Ensures you get the optimal model, not just the final one

    # PERFORMANCE OPTIMIZATION
    fp16=True                        # Use 16-bit floating point instead of 32-bit
                                    # Reduces memory usage by ~50% and speeds up training
                                    # Requires modern GPU with tensor cores for best results
                                    # Set to False if you encounter numerical instability
)

# ============================================================================
# TRAINER: High-level wrapper that handles the entire training process
# ============================================================================

# The Trainer class automates:
# - Training loops and gradient updates
# - Evaluation during training
# - Model saving and checkpointing
# - Logging and progress tracking

trainer = Trainer(
    # MODEL COMPONENTS
    model=peft_model,                # Your LoRA-enhanced model from previous steps
                                    # Already has frozen base weights + trainable LoRA adapters

    args=training_args,              # Training configuration defined above
                                    # Controls all aspects of the training process

    # DATASETS
    train_dataset=train_dataset,     # Your prepared training data
                                    # Should be tokenized and formatted for your task
                                    # Each sample should have input_ids, attention_mask, labels

    eval_dataset=test_dataset,       # Your validation/test dataset
                                    # Used for periodic evaluation during training
                                    # Helps detect overfitting and select best checkpoint

    # TEXT PROCESSING
    tokenizer=tokenizer              # Tokenizer that converts text to model inputs
                                    # Must match the tokenizer used for your base model
                                    # Handles padding, truncation, special tokens
)

# ============================================================================
# EXECUTE TRAINING
# ============================================================================

# Start the training process
# This will:
# 1. Run forward pass through model
# 2. Calculate loss between predictions and labels
# 3. Compute gradients via backpropagation
# 4. Update only the LoRA adapter weights (base model stays frozen)
# 5. Log progress and save checkpoints as configured
# 6. Evaluate on validation set at end of each epoch
trainer.train()

/tmp/ipython-input-175401955.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anandraj001119 (anandraj001119-training-and-placement-cell) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.575600,1.501777


TrainOutput(global_step=1000, training_loss=1.5858620910644532, metrics={'train_runtime': 224.9094, 'train_samples_per_second': 71.14, 'train_steps_per_second': 4.446, 'total_flos': 687392926350336.0, 'train_loss': 1.5858620910644532, 'epoch': 1.0})

In [ ]:
import torch

# Example text to classify
text = "I feel happy today!"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to the same device as the model (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device)
inputs = {name: tensor.to(device) for name, tensor in inputs.items()}


# Make a prediction
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = peft_model(**inputs)

# Get the predicted class (the one with the highest score)
predictions = torch.argmax(outputs.logits, dim=1)

# Get the label text for the predicted class
# You can get the mapping from label id to label text from your dataset
# For the 'emotion' dataset, the order is typically:
# 0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'
label_map = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

predicted_label = label_map[predictions.item()]

print(f"The text '{text}' is classified as: {predicted_label}")

The text 'I feel happy today!' is classified as: joy


In [ ]:
# Evaluate the model on the test dataset
evaluation_results = trainer.evaluate(test_dataset)

# Print the evaluation results
print(evaluation_results)

{'eval_loss': 1.5017768144607544, 'eval_runtime': 5.3459, 'eval_samples_per_second': 374.115, 'eval_steps_per_second': 46.764, 'epoch': 1.0}


In [ ]:
# Save the LoRA adapters
output_dir = "./peft_emotion_classifier"
peft_model.save_pretrained(output_dir)

print(f"LoRA adapters saved to {output_dir}")

LoRA adapters saved to ./peft_emotion_classifier
